In [1]:
import os
path = '/Volumes/hdd/2.public/1.ML/Kaggle/Brazilian_Ecommerce/'

# 라이브러리 불러오기

 * 선행 설치 패키지
     1. folium
     2. nltk

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from matplotlib.gridspec import GridSpec
pd.set_option('display.max_columns', 100)
import plotly.offline as py
import plotly.express as px
import plotly.graph_objs as go
import json
import requests
import folium
from folium.plugins import FastMarkerCluster, Fullscreen, MiniMap, HeatMap, HeatMapWithTime, LocateControl

# Utilities
from viz_utils import *
from custom_transformers import *

# Text prep
import re
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'viz_utils'

# 데이터 불러오기

해당 과업에는 서로 다른 데이터 소스가 있으며, 각각은 전자상거래 판매와 관련된 특정 주제를 나타냅니다.  
파일은 다음과 같습니다.  

- olist_customers_dataset.csv
- olist_geolocation_dataset.csv
- olist_orders_dataset.csv
- olist_order_items_dataset.csv
- olist_order_payments_dataset.csv
- olist_order_reviews_dataset.csv
- olist_products_dataset.csv
- olist_sellers_dataset.csv
- product_category_name_translation.csv


파일들 사이의 관계는 문서에 설명되어 있습니다.  
데이터 세트를 읽고 기초 분석을 시작해보겠습니다. 이 단계는 향후 EDA에서 올바른 결정을 내리는데 많은 도움이 될 것입니다. 

In [ ]:
# 모든 파일 불러오기
olist_customer = pd.read_csv(path + 'olist_customers_dataset.csv')
olist_geolocation = pd.read_csv(path + 'olist_geolocation_dataset.csv')
olist_orders = pd.read_csv(path + 'olist_orders_dataset.csv')
olist_order_items = pd.read_csv(path + 'olist_order_items_dataset.csv')
olist_order_payments = pd.read_csv(path + 'olist_order_payments_dataset.csv')
olist_order_reviews = pd.read_csv(path + 'olist_order_reviews_dataset.csv')
olist_products = pd.read_csv(path + 'olist_products_dataset.csv')
olist_sellers = pd.read_csv(path + 'olist_sellers_dataset.csv')

## 데이터 개요

In [ ]:
# Collections for each dataset
datasets = [olist_customer, olist_geolocation, olist_orders, olist_order_items, olist_order_payments,
            olist_order_reviews, olist_products, olist_sellers]
names = ['olist_customer', 'olist_geolocation', 'olist_orders', 'olist_order_items', 'olist_order_payments',
         'olist_order_reviews', 'olist_products', 'olist_sellers']

# Creating a DataFrame with useful information about all datasets
data_info = pd.DataFrame({})
data_info['dataset'] = names
data_info['n_rows'] = [df.shape[0] for df in datasets]
data_info['n_cols'] = [df.shape[1] for df in datasets]
data_info['null_amount'] = [df.isnull().sum().sum() for df in datasets]
data_info['qty_null_columns'] = [len([col for col, null in df.isnull().sum().items() if null > 0]) for df in datasets]
data_info['null_columns'] = [', '.join([col for col, null in df.isnull().sum().items() if null > 0]) for df in datasets]

data_info.style.background_gradient()

# Grapical Exploration (그래픽 탐색?)
---
그래서 이제 우리는 브라질의 전자상거래로부터 통찰력을 얻기 위해 탐구적인 데이터 분석을 살펴설 것이다. 여기서 목표는 이 세션을 주제로 나누어 각 주제(주문, 고객, 제품, 항목 등)에 대한 그래픽을 탐색하는 것입니다.

브라질의 전자상거래로부터 인사이트를 얻기 위해 탐색적 데이터 분석(EDA)를 진행할 것 입니다.  
여기서 목표는 세션별로 주제를 나누어 각 주제(주문, 고객, 제품, 항목 등)에 대해 탐색한 데이터를 그래픽으로 나타내는 것입니다.  

## Total Orders on E-Commerce
우리는 전자상거래가 세계적 관점에서 실제로 증가하는 추세라는 것을 알고 있습니다.
주문 데이터를 통해 브라질의 전자 상거래 추세에 대해 알아보겠습니다.  

데이터 세트 컬럼을 보면 시간에 따라 구매, 승인, 배송 및 배송 지연과 같이 각각 다른 주문 상태를 볼 수 있습니다. 
먼저, 이데이터 세트에 포함된 주문 상태를 보겠습니다. 

In [ ]:
## single_countplot 대체

plt.figure(figsize=(10,10))
plt.title('주문별 상태', fontsize=15)
sns.countplot(df_orders['order_status'], order=df_orders['order_status'].value_counts().index)

전체 상태중 'delivered'가 가장 많았으며 나머지 상태는 3%에 불과

다음 플롯에서는 구매 주문에 관한 전자 상거래의 증가 추세에 대해 살펴보겠습니다.  
이를 위해 다음 항목에서 **order_purchase_timestamp**에 대한 정보를 추출해야 합니다. 
1. 타임스탬프 컬럼 변환
2. 시간 속성 추출(연/월/일/요일 및 시간)
3. 해당 속성을 통한 전자상거래 추세 파악

In [ ]:
# Changing the data type for date columns
timestamp_cols = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 
                  'order_estimated_delivery_date']
for col in timestamp_cols:
    df_orders[col] = pd.to_datetime(df_orders[col])
    
# Extracting attributes for purchase date - Year and Month
df_orders['order_purchase_year'] = df_orders['order_purchase_timestamp'].apply(lambda x: x.year)
df_orders['order_purchase_month'] = df_orders['order_purchase_timestamp'].apply(lambda x: x.month)
df_orders['order_purchase_month_name'] = df_orders['order_purchase_timestamp'].apply(lambda x: x.strftime('%b'))
df_orders['order_purchase_year_month'] = df_orders['order_purchase_timestamp'].apply(lambda x: x.strftime('%Y%m'))
df_orders['order_purchase_date'] = df_orders['order_purchase_timestamp'].apply(lambda x: x.strftime('%Y%m%d'))

# Extracting attributes for purchase date - Day and Day of Week
df_orders['order_purchase_day'] = df_orders['order_purchase_timestamp'].apply(lambda x: x.day)
df_orders['order_purchase_dayofweek'] = df_orders['order_purchase_timestamp'].apply(lambda x: x.dayofweek)
df_orders['order_purchase_dayofweek_name'] = df_orders['order_purchase_timestamp'].apply(lambda x: x.strftime('%a'))

# Extracting attributes for purchase date - Hour and Time of the Day
df_orders['order_purchase_hour'] = df_orders['order_purchase_timestamp'].apply(lambda x: x.hour)
hours_bins = [-0.1, 6, 12, 18, 23]
hours_labels = ['Dawn', 'Morning', 'Afternoon', 'Night']
df_orders['order_purchase_time_day'] = pd.cut(df_orders['order_purchase_hour'], hours_bins, labels=hours_labels)

# New DataFrame after transformations
df_orders.head()

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(13, 10))

# Axis definition
gs = GridSpec(2, 2, figure=fig)
ax1 = fig.add_subplot(gs[0, :])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1])

# Lineplot - Evolution of e-commerce orders along time 
sns.lineplot(data=df_orders['order_purchase_year_month'].value_counts().sort_index(), ax=ax1, 
             color='darkslateblue', linewidth=2)
ax1.annotate(f'Highest orders \nreceived', (13, 7500), xytext=(-75, -25), 
             textcoords='offset points', bbox=dict(boxstyle="round4", fc="w", pad=.8),
             arrowprops=dict(arrowstyle='-|>', fc='w'), color='dimgrey', ha='center')
ax1.annotate(f'Noise on data \n(huge decrease)', (23, 0), xytext=(48, 25), 
             textcoords='offset points', bbox=dict(boxstyle="round4", fc="w", pad=.5),
             arrowprops=dict(arrowstyle='-|>', fc='w'), color='dimgrey', ha='center')
format_spines(ax1, right_border=False)  
for tick in ax1.get_xticklabels():
    tick.set_rotation(45)
ax1.set_title('Evolution of Total Orders in Brazilian E-Commerce', size=14, color='dimgrey')

# Barchart - Total of orders by day of week
single_countplot(df_orders, x='order_purchase_dayofweek', ax=ax2, order=False, palette='YlGnBu')
weekday_label = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
ax2.set_xticklabels(weekday_label)
ax2.set_title('Total Orders by Day of Week', size=14, color='dimgrey', pad=20)

# Barchart - Total of orders by time of the day
day_color_list = ['darkslateblue', 'deepskyblue', 'darkorange', 'purple']
single_countplot(df_orders, x='order_purchase_time_day', ax=ax3, order=False, palette=day_color_list)
ax3.set_title('Total Orders by Time of the Day', size=14, color='dimgrey', pad=20)

plt.tight_layout()
plt.show()

## E-Commerce Around Brazil

## E-Commerce Impact on Economy 

## Payment Type

# Natural Language Processing
자연어 처리

## Data Understanding

## RegEx

아마 HTML 태그, 개행 문자, 특수 문자 및 데이터 세트의 일부가 될 수 있는 기타 콘텐츠 처리 필요

### 함수 정의
- 적용된 정규식 결과분석하는데 사용될 함수 정의

In [10]:
# 적용된 정규식 결과를 분석하는데 사용될 함수 정의  
def find_patterns(re_patterns, text_list):
    '''
    re_patterns : 검색에 사용할 정규 패턴[type : string]
    text_list : 텍스트 문자열이 있는 목록[type : list]
    
    (반환)
    positions_filename : 다음과 같이 키-값 parse를 갖는 python 사전
    text_idx : [(start_filename1, end_filename1), (start_filename1, end_filename2),...,(start_n,end_fn)]
    '''
    
    # 인수로 전달된 정규식 컴파일
    p=re.compile(re.pattern)
    positions_dict = {}
    i = 0
    for c in text_list:
        match_list = []
        iterator = p.finditer(c)
        for match in iterator:
            match_list.append(match.span())
        control_key = f'Text idx {i}'
        if len(match_list) == 0:
            pass
        else:
            postions_dict[control_key] = match_list
        i += 1
    """
    p = '['R']{0,1}\$[]{0,}\d+(,|\.)\d+'
    patter_dict = find_patterns(p, reviews_breakline)
    print(len(pattern_dict))
    patter_dict
    for idx in [int(c.split(' ')[-1]) for c in list(pattern_dict.keys())]:
    print(f'{reviews_breakline[idx]}\n)
    """
    return postions_dict


## 변환 결과 출력 함수
def print_step_result(text_list_before, text_list_after, idx_list):
    """
    text_list_before : 변환 전 텍스트 내용과 객체 출력 [type:list]
    text_list_after : 변환 후 텍스트 내용과 객체 출력 [tyep:list]
    idx_list : 출력할 인덱스가 있는 객체 [type:list]
    """
    
    # 문자열 예제 반복
    i = 1
    for idx in idx_list:
        print(f"---Text {i} ---\n")
        print(f"Before : \n{text_list_before[idx]}\n")
        print(f"Aftre : \n{text_list_after[idx]}\n")
        i+=1
        
##1. 개행 문자 및 객체 출력(Break Line and Carriage Return)
def re_breaklin(text_list):
    """
    text_list : 준비된 텍스트 콘텐츠 객체 [type : list]
    """
    
    # 정규 표현식 적용
    return [re.sub('[\n\r]', '',r) for r in text_list]

##2. 사이트 및 하이퍼링크 패턴 제거
def re_hiperlinks(text_list):
    """
    text_list : 준비된 텍스트 콘텐츠 객체 [type : list]
    """
    
    # 정규표현식 적용
    patter = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    return [re.sub(pattern, ' link ', r) for r in text_link]

##3. 날짜 제거
def re_dates(text_list):
    """
    text_list : 준비된 텍스트 콘텐츠 객체 [type : list]
    """
    
    pattern = '([0-2][0-9]|(3)[0-1])(\/|\.)(((0)[0-9])|((1)[0-2]))(\/|\.)d{2,4}'
    return [re.sub(pattern, ' data ', r) for r in text_list]

### Breake Line and Carriage Return
개행 문자 및 객체 반환

In [ ]:
# 코멘트 리뷰 목록 생성
reviews = list(df_comments['comment'].values)

# 정규 표현식 적용
reviews_breakline = re_breakline(reviews)
df_comments['re_breakline'] = reviews_breakline

# 결과 확인
print_step_result(reviews, reviews_breakline, idx_list=[48b])

여기서 \r 및 \n 태그를 볼 수 있으며, RegEx를 사용하면 해당 패턴 제거 가능

### Site and Hiperlinks
사이트와 하이퍼링크 패턴 제거

In [ ]:
# 정규 표현식 적용
reviews_hiperlinks = re_hiperlinks(reviews_breakline)
df_comments['re_hiperlinks'] = reviews_hiperlinks

# 결과 확인
print_step_result(reviews_breakline, review_hiperlinks, idx_list=10796, 12782)

### Dates
날짜 전처리

In [ ]:
# 정규 표현식 적용
reviews_dates = re_dates(reviews_hiperlinks)
df_comment['re_dates'] = reviews_dates

# 결과 확인
print_step_result(review_hiperlinks, reviews_dates, idx_list=[26665, 41497, 41674])

### Money

### Numbers

### Negation

### Special Characters

### Additional Whitespaces

## StopWords

## Stemming

## Feature Extraction

### CountVectorizer

### TF-IIDF